In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

In [7]:
with open('onegin.txt', 'r', encoding="utf8") as f:
    text=f.read()


In [9]:
print(text[:400])

He мысля гордый свет забавить,

Вниманье дружбы возлюбя,

Хотел бы я тебе представить

Залог достойнее тебя,

Достойнее души прекрасной,

Святой исполненной мечты,

Поэзии живой и ясной,

Высоких дум и простоты;

Но так и быть – рукой пристрастной

Прими собранье пестрых глав,

Полусмешных, полупечальных,

Простонародных, идеальных,

Небрежный плод моих забав,

Бессонниц, легких вдохновений,

Незр


In [13]:
processed_text = text.lower()


In [14]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 170799
total chars: 98


In [15]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 56920


In [16]:
sentences

['he мысля гордый свет забавить,\n\nвниманье',
 'мысля гордый свет забавить,\n\nвниманье др',
 'ля гордый свет забавить,\n\nвниманье дружб',
 'гордый свет забавить,\n\nвниманье дружбы в',
 'дый свет забавить,\n\nвниманье дружбы возл',
 ' свет забавить,\n\nвниманье дружбы возлюбя',
 'ет забавить,\n\nвниманье дружбы возлюбя,\n\n',
 'забавить,\n\nвниманье дружбы возлюбя,\n\nхот',
 'авить,\n\nвниманье дружбы возлюбя,\n\nхотел ',
 'ть,\n\nвниманье дружбы возлюбя,\n\nхотел бы ',
 '\n\nвниманье дружбы возлюбя,\n\nхотел бы я т',
 'ниманье дружбы возлюбя,\n\nхотел бы я тебе',
 'анье дружбы возлюбя,\n\nхотел бы я тебе пр',
 'е дружбы возлюбя,\n\nхотел бы я тебе предс',
 'ружбы возлюбя,\n\nхотел бы я тебе представ',
 'бы возлюбя,\n\nхотел бы я тебе представить',
 'возлюбя,\n\nхотел бы я тебе представить\n\nз',
 'любя,\n\nхотел бы я тебе представить\n\nзало',
 'я,\n\nхотел бы я тебе представить\n\nзалог д',
 '\nхотел бы я тебе представить\n\nзалог дост',
 'тел бы я тебе представить\n\nзалог достой

In [17]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [18]:
x.shape

(56920, 40, 98)

In [19]:
y.shape

(56920, 98)

In [20]:
model = Sequential()
model.add(LSTM(128, dropout=0.2, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               116224    
_________________________________________________________________
dense (Dense)                (None, 98)                12642     
Total params: 128,866
Trainable params: 128,866
Non-trainable params: 0
_________________________________________________________________


In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)    

In [24]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [25]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
56832/56920 [============================>.] - ETA: 0s - loss: 2.6632****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: ",

а единицами – себя.

мы все глядим в "
,

а единицами – себя.

мы все глядим в правора





xxvii




на поло пола полорой,

в пола поленый боло дора





xxxvi




на поль он поророна,

он поленный полит,

на пола полоро пола

и поло полать соренный просто,

он в пола поло полоной,





xxvii




на поль он в поли праста,

и так поль он пола пола,

и поло полосто серора





xxvi




на поль в полорово на поло





xxvii




на пола поль и прастой,





xxvii




поль он по
----- temperature: 0.5
----- Generating with seed: ",

а единицами – себя.

мы все глядим в "
,

а единицами – себя.

мы все глядим в соль,

как вы веда полу вот вы намо





xxxivii




то полать восерог.





xxii




но прагой полобо,

востари момной това,





xxvv



на сердце позрать и послода;

не всё был дыше полуви послодной

так поскруг простоловал в тате

про вастрена и но сельной нескаждой,

с лет не перед поской девой,

и рак бально ты, простовуть,

но в долько в следал быть,

он соненкий споре сонь

простасть на постал поона

он на на другой и дого вы стара

но
----- temperature: 1.0
----- Generating with seed: "и с чем не связанные сны,

угрозы, толки"
и с чем не связанные сны,

угрозы, толки плед следетвий

с же свеаави (круг я

что в ожим мне: лужак буга:

ты, как и срадные пешку,

я не что душу тель их ругом

но свотроват в нас уумы вер,

не ракуйки, том и бладосный,

хоть она егу.





xxxx




и хоти, уж и воспрой вобриньй,

где домще па танца





xxxixii




и томосллничый скрастью скак.





xxxli




д в там бы на простаяш

чта любву прочновою долинда,

я делый угожены жлей.

----- temperature: 1.2
----- Generating with seed: "и с чем не связанные сны,

угрозы, толки"
и с чем не связанные сны,

угрозы, толкий про вигу.

исьдыат щ

его невестве ждету тепелы

пазней взгурныя режеосйе нен

а изпроззни… что ведне,

о полву дласа ящен;

астрадомкa: осийной чагим…

потолка.а ей: .




……




xvii




вурнистный усуда, его.

«поком скалата ногял, всё – исбсупали.

на позноп яльное ж дрезь!

и злабко быромо мнестей,

кылист убессточив, на снова,

и ли, мисье троперька, девяц

с тока.





iiv




нрут оне в битою стили.

с бечумки на чемикую
56920/56920 [==============================] - 52s 910us/sample - loss: 1.9699
Epoch 8/60
56832/56920 [============================>.] - ETA: 0s - loss: 1.9511****************************************************************************
----- Generating text after Epoch: 7
----- temperature: 0.2
----- Generating with seed: "троем,

и после тайно обесславить

весел"
троем,

и после тайно обесславить

веселые послед простотель,

и все же в приводил на стали,

и татьяна в приводный нестей,

и все была полна последа,

под приводит он себя слезы,

привод ней невольный краст

на всем полна

и в соходной полостой нас

на страсть и так и был сонда,

и то на страсти он отражданье,

приворки страсть и в том она.





xxxvii




в так он с нем на страстви она.





xxxii




и в татьяна страствой полодит.





xxxii




он соходной полна и в ней

на полна в нем не
----- temperature: 0.5
----- Generating with seed: "а душа!.. когда-нибудь

заедем к ним; ты"
а душа!.. когда-нибудь

заедем к ним; ты на лама,

так буртый девячи тоской

подой одном он отровать,

в домится полном он отрал

приметельных долго ль волнова

на страстной превленье, с ней

и солесь он полененый.





xxxxii




как глядительных долго на скоре

в горов не потом от разна.

я дом, что молодой карит жень;

и все умен, но спелый нас вести.





viii




потам своей нас моя дама

в ее в замом девять не плазна,

и с не молв
----- temperature: 1.0
----- Generating with seed: "а душа!.. когда-нибудь

заедем к ним; ты"
а душа!.. когда-нибудь

заедем к ним; тыт жаками

дамалось в сун, страстьм ни петрад,

тас евот п

как зашипевшего евгений,

скузки сголкарзвенным служай

и из ром потвой, иль веледу крое своей.





xxxvii




как тиися сожданной ольсона

печем его берною оте.

под, тосты исбуги респокоие,

в гуукрули скажескенедь муды;

и был кок-ми колга ще забые

с плетые хладным совных,

хоть но разждел на расокож»....


ней страстволи разальнай душою

простудим вечно блакупгли,

скрыльно с пылаю вечный,

и тридод цажтые
----- temperature: 1.2
----- Generating with seed: " неге, в пламени любви,

как зашипевшего"
 неге, в пламени любви,

как зашипевшего вруй карову,

гримы, я ольге язывтицми,

полугим повидралковь пластелся.





iv




мизверик сумтонале.

борог, но чегогуюд прика!

простомания сам и наях

кавал4авлия друг, ит арим.

«сердце греса, изменя

оглазы, вглуше,

им калокитiерь

па аликачида;

благоскает. гляханей, —

вре, не aльвоею радовые

душо весь и, так крим…

разжузн одех ог не хожа;

душная паскорим ее рака.

вновь  озмлебный 
56920/56920 [==============================] - 59

...

струились тишине неостродит

на по, там ты весня прошллста!

стаилекиннадавал глад,

чтоб тесни лужденка татьяна.

она (ночине, без изблезу

к вере нпо мирулано,

что всегди полдева отвстхада

на таблинное комновых,

с белея быя

мое мустено, хренили,

что никво страналежных ону,

трудорой, пуст





xliv




«не ту! писан крихи

как жесвата

друд, верно жерлый,

что в горой столликого;

и умядежнум, обоитомн
56920/56920 [==============================] - 39s 689us/sample - loss: 1.6836
Epoch 46/60
56832/56920 [============================>.] - ETA: 0s - loss: 1.6895****************************************************************************
----- Generating text after Epoch: 45
----- temperature: 0.2
----- Generating with seed: " недостатки кожи.

(«исповедь» ж.-ж. рус"
 недостатки кожи.

(«исповедь» ж.-ж. русской сон!»





xxxii




он от не примтет в отессе стали

и после не страсть на страсть,

и страсть и в как-то всем очей.





xxxii




но волную долги старина,

и должны на по

и все под полне полна.





xxxiii




но все домнут и привленала,

и привычный замечальных друг,

в пере раздела и возна.





xxxii




в воспеленных страстий света,

и все призранный друг,

привод не приводит одно.





xxxix




в то в привычный друг, не привледеть,

и слушала последних 
----- temperature: 0.5
----- Generating with seed: " тани,

и, утренней луны бледней

и треп"
 тани,

и, утренней луны бледней

и трепела последной долго страсть:

он оставил сужденный,

и в только не страстью.





xxxii




то всем не приметлась, моя и смин

после разывул и нескоро

его скрыльственный послушный

с ним и приморули слезы сводй.





xliii




по нею в светренье головой

тебе вечно не свет одно;

и соверчивая толковь,

и правда мои последных речь,

и по мне по томною моей

друзей молодой мой и слодом,

и вольно п
----- temperature: 1.0
----- Generating with seed: " тани,

и, утренней луны бледней

и треп"
 тани,

и, утренней луны бледней

и треправилию толст,

кто грямою но ини кари

затеи хитрости мока

последних былами она

поспойный насиздойх, мойм!

пушу рукапленье… тиши

исстовля, дибный изволенья,

она всталиц соненим сином,

а потослой сотилав

нушей хотствую был свет,

бран, кас кункую являньем

и раскучь пиещенья?»

«ей-нин истрежил, тумансти,

то нечень. находи шеба

до страшенные необед,

на нин соне,[кряд камичусме

задольу, хладно их дам ни чирты,

его нас по тем? небосной,

ты 
----- temperature: 1.2
----- Generating with seed: "увидит ваш суровый взор

затеи хитрости "
увидит ваш суровый взор

затеи хитрости пелум,

он остает тукетнаря,

родаду неб1ы, наблащу,

вартать неватвый ладама,

перес кровься, калуя жико;

поблопится, что вань

сердце нистадамбывал,

совни заметувтевьет.

ковайский душо редится,

в глядит как жь? пофьблена,

овброжатхая; ручит.

какой ли яй бладолкок.

пышел я звалки н сумик

пистыентый наслабленной.»





xlv




ты заблюб, блыждать ихость

друг вздо, буда!. довой,

с игамя в
56920/56920 [==============================] - 40

праглл, пенст
56920/56920 [==============================] - 41s 720us/sample - loss: 1.6728
Epoch 56/60
56832/56920 [============================>.] - ETA: 0s - loss: 1.6725****************************************************************************
----- Generating text after Epoch: 55
----- temperature: 0.2
----- Generating with seed: " пуховый,

нигде ни пятнышка чернил.

он"
 пуховый,

нигде ни пятнышка чернил.

он все это мне полно взгрядым,

и все делать не всем после,

не вздух без него в полно,

и все должно в ней наши мой,

и слюду слушала послова,

и так в полно на наши день;

на девочкий в полно в прость.

но нашей полной своей.





xxxix




всё ним полно, в постепенный,

и всё ним полно в полно,

и всё дамно в ней наших слез,

и все долки, в полно в самом

в городительных полны,

и все подама стра
----- temperature: 0.5
----- Generating with seed: " пуховый,

нигде ни пятнышка чернил.

он"
 пуховый,

нигде ни пятнышка чернил.

он смелы, свой постепий красанья,

его в взд

бывало, в позлузной поэта?

со времянных бругом она

и подажденный любовь:

и в сам сидела в послушный

в свою потому домой, умелый,

в тихо полную с ольбой.





xxxiv




онегин пересладела пристудет;

и все я в отих свое советшьй,

увы! в отрадал сей отаме,

и все ей после в брагами,

и прослу позде тел и наши,

ее домний молодой поэт,

по свете молодой сосед;

она после любовьях на простый.





54








----- temperature: 1.0
----- Generating with seed: "качал таинственный венок.

бывало, в поз"
качал таинственный венок.

бывало, в позделся… потути,

печальной потной он отрадных,

того с тих он прикранных завны

и тень я пред мог, к греза твяеи,

домог римузший лежтел,

пирому с евгений китух,

к ней насводя судьба ж.)





54




пошем позд экей лретских лукама.





xvii




любит: для насладой детстный;

всё тогит он стразатым вздольность,

стих боле палитас ужел долона,

а измаз ль он примернта,

клядит, триет беисть ятри л
----- temperature: 1.2
----- Generating with seed: 